In [1]:
#Need variable to tell write_debug that it can write directly
#to econsole rather than with rootque. Similarly, need a delay
#or second version of root_loop() for when root.mainloop is
#running. Can also use rootque to move things in and out of IO
#functions... rather than wait for kill in root_loop(), can I
#do a callback on rootque like tkinter Var.trace()? Just use
#root_loop() before mainloop, then after, use a trace to deal
#with the result when it arrives?

In [2]:
#Open debug file and write whatever in it, also write to the console.
#Pass[0]: debug data to write
#Set[0]: insert new line in econsole

def write_debug(data) :
    
    data = str.splitlines(data)
    b = ''
    for i in data :
        a = str(datetime.datetime.now()) + ': ' + i + '\n'
        b = b + a
        
        c = i + '\n'

        if runloop :
            rootque.put(c)
        else :
            econsole.insert(tk.END,c)
            econsole.see(tk.END)
            
    if writedbg :
        debug = open(ddir, 'a')
        debug.write(b)
        debug.close()
    
    return()


In [3]:
import os
import time
import datetime
import csv
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image as imgget
from PIL import ImageTk
from concurrent.futures import ThreadPoolExecutor as TPEx
import queue

#These will be moved out to .ini file to customize
qwdir = 'C:/MCOSMOS_V42/QueWay/' #Directory of QueWay folder. RemoteManager needs to point here...
getdir = 'accepted' #Part of directory name for directories to use
writedbg = True #Write debug file
offline = True #Run offline with fake data to test
homecoord = 1000 #starting coord
retrycnt = 1 #number of times to retry RemoteManager comm1 before giving up.
spcnames = ['jobnum','phasenum','controlnum','machnum','samsize','partnum','fullpartnum','phasenam','gridnums']
#controlnum corresponds to SPC control type: CIS = 1, CRP = 2, CFF = 3, None = 4

rask = qwdir + 'REMOTE.ASK'
rans = qwdir + 'REMOTE.ANS'
rmsg = qwdir + 'REMOTE.MSG'
spcdir = qwdir + 'OUT\spc.txt'
ddir = qwdir + 'debug.txt'

proglistnames = ['program','base_path','folder','img']
proglist = pd.DataFrame(columns = proglistnames)
spcdata = pd.DataFrame(columns = spcnames)

In [4]:
root = tk.Tk()
root.title('QueWay')
root.resizable(False, False)

econsole = tk.Text(root, width = 120, height = 8, exportselection = False, wrap = tk.WORD)
econsole.grid(column = 0, row = 2, columnspan = 3, sticky = (tk.N, tk.S, tk.W, tk.E))  

root.update()

rootx = econsole.winfo_width() + 12
rooty = econsole.winfo_height() + 12
padx = (root.winfo_screenwidth() - rootx) / 2
pady = (root.winfo_screenheight() - rooty) / 2
root.geometry(str(rootx) + 'x' + str(rooty) + '+' + str(int(padx)) + '+' + str(int(pady)))
for child in root.winfo_children(): child.grid_configure(padx = 6, pady = 6)

root.update()

rootque = queue.Queue()
runloop = True

os.system("python ./RMFunc.py")
from RMFunc import *
os.system("python ./MiscFunc.py")
from MiscFunc import *

def root_loop() :
    try :
        a = rootque.get(block = False)
        if runloop and a != 'kill' :
            econsole.insert(tk.END,a)
            econsole.see(tk.END)
        if a == 'kill' :
            return()
        root.update()
        time.sleep(0.01)
        root_loop()
    except queue.Empty :
        root.update()
        time.sleep(0.03)
        root_loop()
        
        
def do_offline() :
    #Testing only... fudge data
    a = 'C:\\MCOSMOS_V41\\DATA\\Production\\accepted prog part #0000-6599'
    b = '24'
    c = '491100.A7 PREM RV1.0	C:\\MCOSMOS_V41\\DATA\\Production\\accepted prog part #0000-6599\\PDC5Q70P\\491100.A7.jpg'
    d = '* 491100.A7 PREM RV1.0'
    e = [[a,b,c,d]]
    write_debug('Used fake data because offline = True.\n')
    
    return(e)


#Get HELP List (test out RM connect and print list of available commands to debug)
reply = TPEx(max_workers=1).submit(comm1,'HELP')
root_loop()

if reply.result() == [] :
    write_debug('Failed to communicate with RemoteManager. Please review configuration. Switching to offline demo mode.\n')
    offline = True
else :
    write_debug('Successfully communicated with RemoteManager!\n')

if offline :
    pathfiles = do_offline()
else :
    #Find "getdir" group in list of directories
    dirs = TPEx(max_workers=1).submit(comm1,'GET_DIR_LIST')
    root_loop()

    okpaths = []
    for a in dirs.result() :
        b = a.find(getdir)
        if b != -1 :
            okpaths = okpaths + [a]
            write_debug('Found path: ' + str(a) + ' containing: ' + getdir + '\n')

    #Change to each "OK" dir and get part list there
    pathfiles = []
    for a in okpaths :
        TPEx(max_workers=1).submit(comm1,'CHANGE_PART_DIR\n' + a + '\n')
        root_loop()
        #Add a "Check Part Dir" here to make sure it's switched?
        reply = TPEx(max_workers=1).submit(comm1,'GET_PART_LIST3\n')
        root_loop()
        pathfiles = pathfiles + [reply.result()]

    if len(pathfiles) == 0 :
        write_debug('Did not get any good paths! Switching to offline demo.\n')
        offline = True
        pathfiles = do_offline()
        
#Split lists of programs into |program|base path|folder|jpg name|etc (tack "OK" on first as fake separator)
#Todo: support part programs that are in part programs. Right now assumes one program = one part program
#(skips every other line). If more than one part program, it would get all kinda mixed up.

pathfiles[0] = ['OK'] + pathfiles[0]    
a = []
tick = 0
for i in range(len(pathfiles)) :
    for j in range(len(pathfiles[i])) :
        l = pathfiles[i][j]
        
        if l == 'OK' :
            k = 0
        
        if k > 2 and tick == 0 :
            a = a + [l]
            tick = 1
        else :
            tick = 0
        
        k += 1

store = pd.Series(a)
write_debug('Found ' + str(len(a)) + ' program paths.\n')

#Break out Series into DataFrame

if len(a) != 0 :
    a = store.str.rsplit('\\', expand = True, n = 2)
    b = pd.Series(a[0][:]).str.split('\t', expand = True)
    c = a.drop(columns = [0])
    e = pd.concat([b,c], axis = 1)
    e.columns = proglistnames
    proglist = proglist.append(e)


#proglist is the DataFrame containing all available programs (and their info)
#proglist = d

if offline :
    #Testing only... more fudge data based on fudge data.
    tick = 0
    tock = 0
    for i in range(50) :
        if tick == 0 :
            fase = 'PREM'
        elif tick == 1 :
            fase = 'SINT'
        elif tick == 2 :
            fase = 'CALI'
        else :
            fase = 'VEMB'
        i += 1
        if i <= 9 :
            i = '0' + str(i)
        else :
            i = str(i)
        a = '4' + str(tock) + '11' + i + '.A7 ' + fase + ' RV0.1'
        b = proglist['base_path'].iloc[0]
        c = proglist['folder'].iloc[0]
        d = proglist['img'].iloc[0]
        entry = pd.DataFrame([[a,b,c,d]], columns = ('program','base_path','folder','img'))
        proglist = pd.concat([proglist,entry])
        tick += 1
        if tick == 4 :
            tick = 0
        tock += 1
        if tock == 10 :
            tock = 0

In [5]:
#Show the thumbnail image when clicked
#Get[0]: 'progimg' = physical path to image
#Return[0]: ()

#def show_img() :
#    a = progimg.get()
#    b = imgget.open(a)
#    b.show()
#    write_debug('Displayed large view of: ' + a.rsplit('\\', 1)[1] + '.\n')
#    b.close()
    
#    return()

def show_button_img(button_group) :
    
    strvar = button_group[1]
    
    b = int(strvar.get())
    c = proglist['base_path'].iloc[b] + '\\' + proglist['folder'].iloc[b] + '\\' + proglist['img'].iloc[b]
    d = imgget.open(c)
    d.show()
    write_debug('Displayed large view of: ' + c.rsplit('\\', 1)[1] + ' from grid button.\n')
    d.close
    
    return()

def tog_console(*args) :
    
    a = contog.get()
    
    if a :
        econsole.grid_remove()
        contog.set(False)
        write_debug('Hid the console.\n')
    else :
        econsole.grid()
        contog.set(True)
        write_debug('Unhid the console.\n')
        
    return()

def num_valid(entry,length) :
    
    isnum = entry.isdigit()
    if isnum :
        val = entry
    else :
        val = ''
        for i in entry :
            isnum = i.isdigit()
            if isnum :
                val = val + i
    
    if len(val) > length :
        val = val[:length]
    
    write_debug('Checked \"' + entry + '\" is numeric and len <= ' + str(length) + '. Returned '+ val + '.\n')
    return(val)

#Fill the Job Number and Phase Number fields from the barcode when entry in the barcode field happens
#Pass[0]: *args, not used
#Get[0]: "barcode" entry
#Set[0]: "jobnum" based on first 6 digits of "barcode"
#Set[1]: "phasenum" based on next 2 digits of "barcode"
#Return[0]: ()

def bar_entry(*args) :

    code = barcode.get()
    get = num_valid(code,13)
    barcode.set(get)
    a = get[:6]
    b = get[6:8]
    jobnum.set(a)
    phasenum.set(b)
    
    write_debug('Got barcode entry: ' + get + '.\n')
    write_debug('Set job number to: ' + str(a) + ' and set phase number to: ' + str(b) + '.\n')
    return()

def job_entry(*args) :

    a = jobnum.get()
    get = num_valid(a,6)
    jobnum.set(get)
    
    if len(get) > 2 :
        write_debug('Got job number entry: ' + get + '.\n')
    return()

def phase_entry(*args) :

    a = phasenum.get()
    get = num_valid(a,2)
    phasenum.set(get)
    
    if len(get) > 0 :
        write_debug('Got phase number entry: ' + get + '.\n')
    return()

def mach_entry(*args) :

    a = machnum.get()
    get = num_valid(a,4)
    machnum.set(get)
    
    if len(get) > 2 :
        write_debug('Got machine number entry: ' + get + '.\n')
    return()

def sample_entry(*args) :
    
    hadfoc = samhadfoc.get()
    e = samsize.get()
    print('arg ' + str(args[0]))
    print('had foc ' + str(hadfoc))
    
    if args[0] == 'set' and not hadfoc :
        e = str(int(e) + 1)
        samsize.set(e)
        write_debug('Sample Size set to: ' + e + '.\n')
    
    elif args[0] == 'del' and not hadfoc :
        e = str(int(e) - 1)
        samsize.set(e)
        write_debug('Sample Size set to: ' + e + '.\n')
    
    else :
        get = num_valid(e,2)
        samsize.set(get)
        samhadfoc.set(True)
        if len(get) > 0 :
            write_debug('Got user sample size entry: ' + get + '.\n')
    
    print('focus is: ' + str(root.focus_get()))
    return()

#Get the associated program/s from part number entry. List box shows available
#programs, then user chooses one. Match "proglist" entry in listbox list so
#it can be referenced externally to subprogram.
#Pass[0]: *args, not used
#Get[0]: "partfind" entry
#Set[0]: "proginfo" list based on match
#Set[1]: "proginforef" csv list of "proginfo" reference to "proglist" entry
#Return[0]: ()

def match_partprog(*args) :

    part = str(partfind.get())
    a = proglist[proglist['program'].str.startswith(part, na = False)]
    write_debug('Found ' + str(len(a)) + ' programs containing \"' + part + '\".\n')
    
    #don't bother to list if we have more than a few entries
    if len(a) > 30 :
        proginfo.set('')
        selectprog.set('')
        change_state(eproglist,'disabled')
        return()
    
    #Make a string list of "val" "val2" "val3" for listbox and matching csv list for the proglist entry.
    #ToDo : rewrite using insert etc for list box, move proglist find to happen on listbox select
    #... get rid of proginforef...
    b = ''
    f = ''
    for i in range(len(a)) :
        g = np.where(proglist['program'] == a['program'].iloc[i])[0]
        f = f + str(g[0]) + ','
        d = str(a['program'].iloc[i])
        b = b + '\"' + d + '\" '
        write_debug(d + '\n')

    proginfo.set(b)
    proginforef.set(f)
    change_state(eproglist,'normal')
    
    return()


#Do action on listbox select, set imgthumb, and use the current selection to return the proglist
#reference as "selectprog".
#Get[0]: Currently selected program in "eproglist"
#Get[1]: "proginforef" = list of numeric reference to proglist matching entries in "proginfo"
#Set[0]: "selectprog" = numeric reference to specific prog in proglist
#Set[1]: "progimg" = path to currently selected program image
#Set[2]: "imgthumb" assigned image for selected program
#Return[0]: ()

def listbox_select() :
    #Use selected item in listbox and match it to the ref. values for iloc in proglist from proginforef
    #Set selectprog to int = reference iloc in proglist of selected program
    a = int(eproglist.curselection()[0])
    b = proginforef.get().split(sep = ',')
    c = int(b[a])
    selectprog.set(c)
    d = str(proglist['program'].iloc[c])
    write_debug(d + ' is currently selected.\n')
    
    #Set selection image path
    e = proglist['base_path'].iloc[c] + '\\' + proglist['folder'].iloc[c] + '\\' + proglist['img'].iloc[c]
    progimg.set(e)
    write_debug('Image Thumb set to: ' + e + '\n')

    #parse program name into partnum, fullpartnum, and phasename.
    f = d.split(sep = '.')
    
    g = f[0][-6:]
    partnum.set(g)
    
    h = g + '.' + f[1][:3].strip()
    fullpartnum.set(h)
    
    f = d.split(sep = ' ')
    for i in f :
        if len(i) == 4 and i.isalpha() :
            phasenam.set(i)
            j = i
            
    write_debug('Set part#: ' + g + ', full part#: ' + h + ' and phase name: ' + j + '.\n')
    
    ebarcode.focus()
    return()

#Set grid button to current selection. Checks first if there is a listbox selection.
#Pass[0]: button group
#Pass[0][0]: Button object
#Pass[0][1]: Button StringVar
#Pass[0][2]: Button Number
#Get[0]: 'progimg' = physical path to image
#Set[0]: button.image to selected program image
#Set[1]: button StringVar = proglist reference number
#Return[0]: ()

def set_button(button_group) :
    button = button_group[0]
    strvar = button_group[1]
    butnum = button_group[2].get()
    write_debug('Button #' + str(butnum) + ' was pressed.\n')
    
    if strvar.get() != '' :
        write_debug('Button is already set to: ' + proglist['program'].iloc[int(strvar.get())] + '.\n')
        show_button_img(button_group)
        return()

    if eproglist.curselection() :
        a = progimg.get()
        b = imgget.open(a)
        wsize = button.winfo_width() - 6
        hsize = button.winfo_height() - 6
        b.thumbnail((wsize,hsize),imgget.ANTIALIAS)
        b = ImageTk.PhotoImage(b)
        button.image = b
        button.config(image = b, width = wsize, height = wsize)
        strvar.set(selectprog.get())
        write_debug('Set to part: ' + proglist['program'].iloc[int(selectprog.get())] + ' and '
            + 'image set to: ' + a.rsplit('\\', 1)[1] + '.\n')
        
        c = holdlist.get()
        if len(c) == 0 :
            c = str(butnum)
        else :
            c = c + '-' + str(butnum)

        holdlist.set(c)
        sample_entry('set')
                
        write_debug('Now holding this/these buttons: ' + c + '.\n')
        
    else :
        write_debug('No program was selected to set.\n')
   
    return()


#Clear the grid button/reset it.
#Pass[0]: button group
#Pass[0][0]: Button object
#Pass[0][1]: Button StringVar
#Pass[0][2]: Button Number
#Set[0]: button.image to default (grid) image
#Set[1]: button StringVar = " "
#Return[0]: ()

def clear_button(button_group) :
    button = button_group[0]
    strvar = button_group[1]
    butnum = button_group[2].get()
    
    state = str(button['state'])
    if state == 'disabled' :
        write_debug('Button #' + str(butnum) + ' was not cleared because it is disabled.\n')
        return()

    write_debug('Button #' + str(butnum) + ' was cleared.\n')
    a = imgget.open(qwdir + 'grid.jpg')
    wsize = button.winfo_width() - 6
    hsize = button.winfo_height() - 6
    a.thumbnail((wsize,hsize),imgget.ANTIALIAS)
    a = ImageTk.PhotoImage(a)
    button.image = a
    button.config(image = a, width = wsize, height = wsize)
    strvar.set('')
    
    b = holdlist.get().split(sep = '-')
    c = ''
    if b[0] != '' :
        for i in b :
            if int(i) != butnum :
                if len(c) == 0 :
                    c = i
                else :
                    c = c + '-' + str(i)
    
    holdlist.set(c)
    sample_entry('del')
    
    if len(c) == 0 :
        write_debug('Now holding no buttons.\n')
    else :
        write_debug('Now holding this/these buttons: ' + c + '.\n')
        
    return()


def add_que() :
    
    hold = str(holdlist.get())
    gridnums.set(hold)
    
    #Columns: [jobnum, phasenum, controlnum, machnum, samsize, partnum, fullpartnum, phasenam,gridnums]
    global spcdata
    e = [[jobnum.get(),phasenum.get(),controlnum.get(),machnum.get(),samsize.get(),partnum.get(),fullpartnum.get(),phasenam.get(),gridnums.get()]]
    
    queok = True
    g = 0
    for i in e[0] :
        if i == '' :
            queok = False
            write_debug('NG! ' + spcnames[g] + ': ' + str(i) + '.\n')
        else :
            write_debug('OK! ' + spcnames[g] + ': ' + str(i) + '.\n')
        g += 1
    
    if queok :

        b = ''
        for i in hold.split(sep = '-') :
            change_state(buttlist[int(i)][0],'disabled')
            if b == '' :
                a = int(buttlist[int(i)][1].get())
                b = proglist['program'].iloc[a]

        
        d = b + '>' + hold
        c = d + '>FRESH'
        
        equelist.insert(tk.END,c)
        holdlist.set('')
        
        write_debug('Passed hold (' + d + ') to que and set buttons to disabled.\n')
        
        e = pd.DataFrame(e, index = [d], columns = spcnames)
        spcdata = spcdata.append(e)
    
    else :
        gridnums.set('')
    
    
    #what was I trying to do with this?
    #    if equelist.curselection() == '' :
    #        equelist.activate(0)
    
    return()


def del_que(a) :
    
    b = str(equelist.get(a))
    b = b.split(sep = '>')
    for i in b[1].split(sep = '-') :
        change_state(buttlist[int(i)][0],'normal')
        clear_button(buttlist[int(i)])
        
    #ToDo: redundant much?
    b = equelist.get(a)
    equelist.delete(a)
    b = b.split(sep = '>')[:-1]
    
    global spcdata
    spcdata = spcdata.drop(index = [b])
    
    write_debug('Que selection #' + str(a) + '(' + b + ') was deleted.\n')

    return()


def que_check(*args) :
    
    a = equelist.size()
    b = playtog.get()
    
    if a == 0 :
        change_state(edel,'disabled')
        change_state(eplay,'disabled')
    elif not b :
        change_state(edel,'normal')
        change_state(eplay,'normal')

    write_debug('Que is: ' + str(equelist.get(0,tk.END)) + '.\n')
    
    return()


def hold_check(*args) :
    
    a = len(holdlist.get())

    if a == 0 :
        change_state(eproglist,'normal')
        change_state(eque,'disabled')
    else :
        change_state(eproglist,'disabled')
        change_state(eque,'normal')
    
    return()


def change_state(obj,state) :
    
    a = state
    b = str(obj['state'])
    
    try :
        c = str(obj.config().get('text')[-1]) + ' '
    except :
        c = ''
    
    if a != b :
        obj.config(state = a)
        write_debug('The ' + c + 'object state was switched to: ' + a + '.\n')

    return()


def que_update(loc,state) :

    a = equelist.get(loc).split(sep = '>')
    b = a[0] + '>' + a[1]
    done = b + '>' + state

    c = playtog.get()

    if c :
        equelist.delete(loc)
        equelist.insert(loc, done)
        write_debug('Changed ' + b + '>' + a[2] + ' to ' + done + '.\n')
    else :
        write_debug('Que was stopped before changing ' + b + '>' + a[2] + ' to ' + done + '.\n')

    return()


def play_que(*args) :
    
    if equelist.size() == 0 :
        write_debug('Nothing in Que to run...\n')
        playtog.set(False)
    
    tog = playtog.get()
    
    if tog :
        
        change_state(eplay,'disabled')
        change_state(epause,'normal')
        change_state(edel,'disabled')

        run = runinfo.get()
        write_debug('Que is running. Que run state: ' + run + '.\n')
        quenext = False

        quenow = quefrom.get()
        playwat = equelist.get(quenow)
        a = playwat.split(sep = '>')
        
        if a[0] == '' :
            quenow = 0
            playwat = equelist.get(quenow)
            a = playwat.split(sep = '>')
            write_debug('Looping to top of que.\n')
        
        if a[2] != 'DONE' :
            if run == 'READY' :
                que_update(quenow,'ACTIVE')
                d = a[1].split(sep = '-')
                e = int(buttlist[int(d[0])][1].get())
                f = proglist['program'].iloc[e]
                g = proglist['base_path'].iloc[e]

                reply = TPEx(max_workers=1).submit(comm1,'CHANGE_PART_DIR\n' + g + '\n')
                root_loop()

                if len(reply.result()) != 0 :
                    if reply.result()[0] == 'OK' :

                        spc = spcdata.loc[playwat]
                        write = []
                        index = 0
                        for i in spc :
                            varname = spcnames[index]
                            write = write + [varname + '>' + str(i)]
                            index += 1
                        write_file(write,spcdir,True)

                        msg = 'EXECUTE_PATH_PART_PROGRAM \n'
                        msg = msg + g + '\n'
                        msg = msg + f + '\n' + f + '\n' + str(0) + '\n' + str(0) + '\n'
                        msg = msg + str(homecoord) + '\n' + str(len(d)) + '\n'

                        reply = TPEx(max_workers=1).submit(comm1,msg)
                        root_loop()

                        if len(reply.result()) != 0 :
                            if reply.result()[0] == 'OK' :
                                que_update(quenow,'RUNNING')
                                runinfo.set('RUN')
                            else :
                                que_update(quenow,'ERROR')
                                write_debug('Got ' + reply + ' instead of \"OK\".\n')
                        else :
                            que_update(quenow,'ERROR')
                            quenext = True
                    else :
                        que_update(quenow,'ERROR')
                        write_debug('Got ' + reply + ' instead of \"OK\".\n')
                else :
                    que_update(quenow,'ERROR')
                    quenext = True

            elif run == 'RUN' :
                g = TPEx(max_workers=1).submit(read_file,(rmsg,True))
                root_loop()

                if len(g.result()) != 0 :
                    runinfo.set(g[0])

                write_debug('Currently running ' + a[0] + '.\n')

            elif run == 'PPEND' :
                que_update(quenow,'DONE')
                write_debug(a[0] + ' ran successfully.\n')
                quenext = True

            elif run == 'PPERR' :
                que_update(quenow,'ERROR')
                write_debug(a + ' did not run successfully!\n')
                quenext = True

            else :
                write_debug('Reply ' + run[0] + 'is unknown! Stopping the que.\n')
                runinfo.set('READY')
                playtog.set(False)
        
        else :
            quenext = True
            write_debug('Congrats, ' + playwat + ' is already DONE!\n')
        
        if quenext == True :
            quenow += 1
            playwat = quefrom.set(quenow)
            write_debug('Starting next in que: ' + playwat + ' .\n')
            runinfo.set('READY')

        if offline == True :
            t = 500
        else :
            t = 50
        root.after(t, lambda b=b : play_que())
        
    else :
        change_state(epause,'disabled')
        if equelist.size() != 0 :
            change_state(eplay,'normal')
            change_state(edel,'normal')
        write_debug('Que is not running.\n')
        
    return()


#3 panes/frames in root for entry area, button grid area, and que/run area
entryf = ttk.Frame(root)
entryf.grid(column = 0, row = 0, sticky = (tk.N, tk.E, tk.W))

buttonf = ttk.Frame(root)
buttonf.grid(column = 1, row = 0, rowspan = 2, sticky = (tk.N, tk.E, tk.W))

quef = ttk.Frame(root)
quef.grid(column = 2, row = 0, rowspan = 2, sticky = (tk.N, tk.E, tk.W))

#barcode = StringVar()
#jobnum = StringVar()
#phasenum = StringVar()
#controlnum = IntVar() - selection variable for radiobuttons CIS/CRP/CFF/None
#machinenum = StringVar()
#samsize = StringVar() - lot sample size
#partnum = StringVar()
#fullpartnum = StringVar()
#phasenam = StringVar()
#gridnums = StringVar()

#partfind = StringVar() - part program search box (by part number)
#selectprog = StringVar() - what is currently selected in prog select list
#proginfo = StringVar() (listvar) - list of prog names based on prog number entry
#proginforef = StringVar() - what proglist number matches each proginfo entry
#progimg = StringVar() - path of image matching current selection
#holdlist = StringVar() - set of buttons for current selection (before que)
#quelist = StringVar() (listvar) - list of que'd programs to run
#contog = BooleanVar() - console toggle on or off
#playtog = BooleanVar() - que play/pause on or off
#runinfo = StringVar() - status of que (READY,RUN>[prog],PPEND,PPERR)
#quefrom = IntVar() - what is the current que to run (if error in [0], run [1])
#samhadfoc = BooleanVar() - set when sample entry has gotten focus to stop using set buttons for qty.

blank = ttk.Label(entryf, text = '')
blank.grid(column = 0, row = 0, columnspan = 2)

partfind = tk.StringVar()
lpartfind = ttk.Label(entryf, text = 'Part Number')
lpartfind.grid(column = 0, row = 1, sticky = (tk.E))
epartfind = ttk.Entry(entryf, textvariable = partfind)
epartfind.grid(column = 1, row = 1, sticky = (tk.E, tk.W))
partfind.trace('w', match_partprog)

#list of programs that match current search, selection in this list is whatever program to que
#Todo: change this to set correctly like the que list using get/put instead of writing to the listvar

selectprog = tk.StringVar()
progimg = tk.StringVar()
proginfo = tk.StringVar()
proginforef = tk.StringVar()
eproglist = tk.Listbox(entryf, listvariable = proginfo, state = 'disabled', exportselection = False, width = 30)
eproglist.grid(column = 0, row = 2, columnspan = 2, sticky = (tk.E, tk.W))
eproglist.bind('<<ListboxSelect>>', lambda a=a : listbox_select())

blank = ttk.Label(entryf, text = '')
blank.grid(column = 0, row = 3, columnspan = 2)

#Todo: make these configurable in .ini to add/remove, and parse contents.
partnum = tk.StringVar()
fullpartnum = tk.StringVar()
phasenam = tk.StringVar()
gridnums = tk.StringVar()

barcode = tk.StringVar()
lbarcode = ttk.Label(entryf, text = 'Barcode')
lbarcode.grid(column = 0, row = 6, sticky = (tk.W))
ebarcode = ttk.Entry(entryf, textvariable = barcode)
ebarcode.grid(column = 0, row = 7, sticky = (tk.W, tk.E))
barcode.trace('w', bar_entry)

jobnum = tk.StringVar()
ljobnum = ttk.Label(entryf, text = 'Job Number')
ljobnum.grid(column = 0, row = 8, sticky = (tk.W))
ejobnum = ttk.Entry(entryf, textvariable = jobnum)
ejobnum.grid(column = 0, row = 9, sticky = (tk.E, tk.W))
jobnum.trace('w', job_entry)

machnum = tk.StringVar()
lmachnum = ttk.Label(entryf, text = 'Machine Number')
lmachnum.grid(column = 0, row = 10, sticky = (tk.W))
emachnum = ttk.Entry(entryf, textvariable = machnum)
emachnum.grid(column = 0, row = 11, sticky = (tk.E, tk.W))
machnum.trace('w', mach_entry)

phasenum = tk.StringVar()
lphasenum = ttk.Label(entryf, text = 'Phase Number')
lphasenum.grid(column = 1, row = 8, sticky = (tk.W))
ephasenum = ttk.Entry(entryf, textvariable = phasenum)
ephasenum.grid(column = 1, row = 9, sticky = (tk.E, tk.W))
phasenum.trace('w', phase_entry)

#This is set to not take tab-focus so uses # of set buttons until user gives it focus.
#Then it stops updating from buttons and uses whatever user entered.
samsize = tk.StringVar()
samsize.set('0')
samhadfoc = tk.BooleanVar()
samhadfoc.set(False)
lsamsize = ttk.Label(entryf, text = 'Sample Size')
lsamsize.grid(column = 1, row = 10, sticky = (tk.W))
esamsize = ttk.Entry(entryf, textvariable = samsize, takefocus = 0)
esamsize.grid(column = 1, row = 11, sticky = (tk.E, tk.W))
samsize.trace('w', sample_entry)

#ToDo : make this frame tab-into-able, but not each button. Use arrows to change selected button.
controlf = ttk.Frame(entryf)
controlf.grid(column = 1, row = 6, rowspan = 2, sticky = (tk.S))
controlnum = tk.IntVar()
controlnum.set(2)
econa = ttk.Radiobutton(controlf, text = 'CIS', width = -6, value = 1, variable = controlnum)
econa.grid(column = 0, row = 0, sticky = (tk.E, tk.W))
econb = ttk.Radiobutton(controlf, text = 'CRP', width = -6, value = 2, variable = controlnum)
econb.grid(column = 0, row = 1, sticky = (tk.E, tk.W))
econc = ttk.Radiobutton(controlf, text = 'CFF', width = -6, value = 3, variable = controlnum)
econc.grid(column = 1, row = 0, sticky = (tk.E, tk.W))
econd = ttk.Radiobutton(controlf, text = 'None', width = -6, value = 4, variable = controlnum)
econd.grid(column = 1, row = 1, sticky = (tk.E, tk.W))

#Generate 5x6 rows with buttons. Bake in the lambda commands on creation so each button is it's own reference.
#Make buttlist with each "button" as [button obj, stringvar, button num intvar]
#Todo : make this configurable shape and quantity, with .ini vars to define grid and XY offsets

buttlist = []
b = 0
c = 12
for a in range(30) :
    strvar = tk.StringVar()
    buttonnum = tk.IntVar()
    
    button = tk.Button(buttonf, textvariable = strvar)
    
    d = imgget.open(qwdir + 'grid.jpg')
    d.thumbnail((80,80),imgget.ANTIALIAS)
    d = ImageTk.PhotoImage(d)
    button.image = d
    button.config(image = d)
    
    button.grid(column = b, row = (c - 1))
    buttonnum.set(a)
    
    d = [button,strvar,buttonnum]
    
    button['command'] = lambda d = d : set_button(d)
    #Have to set b=b because bind always passes the action?
    button.bind('<3>', lambda b = b, d = d : clear_button(d))
    button.bind('<Double-1>', lambda b = b, d = d : show_button_img(d))
    
    buttlist = buttlist + [d]
    
    label = ttk.Label(buttonf, text = 'Grid #' + str(a + 1))
    label.grid(column = b, row = c)
    
    if a <= 5 :
        blank = ttk.Label(buttonf, text = '')
        blank.grid(column = b, row = 0)
        blank = ttk.Label(buttonf, text = '')
        blank.grid(column = b, row = 13)
    
    b += 1
    if b > 4 :
        b = 0
        c -= 2

blank = ttk.Label(quef, text = '')
blank.grid(column = 0, row = 0, columnspan = 2)        

#after select prog, click a grid button(s) to 'hold' them as a group, then press que to add that 
#set to the queue list. spcdata will be matched to each que with index set as prog name>grid#-grid#

eque = ttk.Button(quef, text = 'Queue', state = 'disabled', command = lambda a=a : add_que())
eque.grid(column = 0, row = 6, sticky = (tk.E, tk.W))

edel = ttk.Button(quef, text = 'Delete', state = 'disabled', command = lambda a=a : del_que(int(equelist.curselection()[0])))
edel.grid(column = 1, row = 6, sticky = (tk.E, tk.W))

holdlist = tk.StringVar()
holdlist.trace('w', hold_check)
quelist = tk.StringVar()
quelist.trace('w', que_check)
equelist = tk.Listbox(quef, listvariable = quelist, exportselection = False, width = 40)
equelist.grid(column = 0, row = 7, columnspan = 2, sticky = (tk.E, tk.W))

#Play and Pause buttons for running the programs out of the queue.

playtog = tk.BooleanVar()
playtog.set(False)
runinfo = tk.StringVar()
runinfo.set('READY')
quefrom = tk.IntVar()
quefrom.set(0)
epause = ttk.Button(quef, text = 'Pause', state = 'disabled', command = lambda : playtog.set(False))
epause.grid(column = 0, row = 8, sticky = (tk.E, tk.W))
eplay = ttk.Button(quef, text = 'Play', state = 'disabled', command = lambda : playtog.set(True))
eplay.grid(column = 1, row = 8, sticky = (tk.E, tk.W))
playtog.trace('w', play_que)

blank = ttk.Label(entryf, text = '')
blank.grid(column = 0, row = 9, columnspan = 2)

contog = tk.BooleanVar()
contog.set(True)
econtog = ttk.Button(quef, text = 'Console', command = lambda a=a : tog_console())
econtog.grid(column = 0, row = 11, sticky = (tk.E, tk.W))

equit = ttk.Button(quef, text = 'Quit', command = root.destroy)
equit.grid(column = 1, row = 11, sticky = (tk.E, tk.W))

#get first console data and load into new console

save = econsole.get('1.0',tk.END + '-1c')
econsole.destroy()
econsole = tk.Text(quef, width = 40, height = 15, exportselection = False, wrap = tk.NONE)
econsole.grid(column = 0, row = 12, columnspan = 2, sticky = (tk.N, tk.W, tk.E))
for i in save :
    econsole.insert(tk.END,i)
econsole.see(tk.END)

#define weights of columns/rows for window re-sizing, and padding, min window size etc.
    
grids = [root, entryf, buttonf, quef]

for i in grids :
    for j in range(i.grid_size()[0]): i.columnconfigure(j, weight = 1)
    for j in range(i.grid_size()[1]): i.rowconfigure(j, weight = 0)

root.columnconfigure(1, weight = 4)
    
for child in entryf.winfo_children(): child.grid_configure(padx = 4, pady = 4)
for child in buttonf.winfo_children(): child.grid_configure(padx = 1, pady = 1)
for child in quef.winfo_children(): child.grid_configure(padx = 4, pady = 4)

root.minsize(1280,720)
root.maxsize(root.winfo_screenwidth()-10,root.winfo_screenheight()-10)
root.geometry('1280x720')

runloop = False
epartfind.focus()
root.mainloop()

#Change imgthumb and grid buttons from buttons to something else and use a bind on click?

arg set
had foc False
arg PY_VAR13
had foc False
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg set
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg set
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg set
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!entry4
focus is: .!frame.!entry4
arg set
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!listbox
focus is: .!frame.!listbox
arg set
had foc True
arg PY_VAR13
had foc True
focus is: .!frame.!listbox
focus is: .!frame.!listbox


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vernz\AppData\Local\Programs\Python\Python37\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-64df8760943a>", line 569, in play_que
    write_debug('Starting next in que: ' + playwat + ' .\n')
TypeError: can only concatenate str (not "NoneType") to str


arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame3.!button2
focus is: .!frame3.!button2
arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame3.!button2
focus is: .!frame3.!button2
arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame3.!button2
focus is: .!frame3.!button2


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vernz\AppData\Local\Programs\Python\Python37\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-64df8760943a>", line 760, in <lambda>
    edel = ttk.Button(quef, text = 'Delete', state = 'disabled', command = lambda a=a : del_que(int(equelist.curselection()[0])))
  File "<ipython-input-5-64df8760943a>", line 389, in del_que
    spcdata = spcdata.drop(index = [b])
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\frame.py", line 3697, in drop
    errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\generic.py", line 3111, in drop
    obj = obj._drop_axis(labels, axis, level=level, errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\generic.py", line 3143, in _drop_axis
    new_axis = axis.drop(labels, errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\l

arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame3.!button2
focus is: .!frame3.!button2
arg del
had foc True
arg PY_VAR13
had foc True
focus is: .!frame3.!button2
focus is: .!frame3.!button2


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vernz\AppData\Local\Programs\Python\Python37\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-64df8760943a>", line 760, in <lambda>
    edel = ttk.Button(quef, text = 'Delete', state = 'disabled', command = lambda a=a : del_que(int(equelist.curselection()[0])))
  File "<ipython-input-5-64df8760943a>", line 389, in del_que
    spcdata = spcdata.drop(index = [b])
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\frame.py", line 3697, in drop
    errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\generic.py", line 3111, in drop
    obj = obj._drop_axis(labels, axis, level=level, errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\lib\site-packages\pandas\core\generic.py", line 3143, in _drop_axis
    new_axis = axis.drop(labels, errors=errors)
  File "c:\users\vernz\pythenv\64bitvanilla\l